In [ ]:
#!pip install nltk

In [ ]:
#!pip install wordcloud

In [ ]:
import nltk

In [ ]:
#nltk.download()

In [ ]:
import pandas as pd,numpy as np
import matplotlib.pyplot as plt

In [ ]:
reviews = pd.read_csv("K8 Reviews v0.2.csv")

In [ ]:
reviews.head()

This dataset is scrapped from amazon about Lenevo K8 mobile phones.While the users gave the reviews, they also give the ratings
- Ratings 1,2,3 -> negitive sentiment has a value 0 in the sentiment column
- Ratings 4,5 -> positive sentiment has a value 1 in the sentiment column

In [ ]:
reviews.sentiment.value_counts()

In [ ]:
reviews.shape

In [ ]:
from wordcloud import WordCloud

In [ ]:
reviews_combined = " ".join(reviews.review.values)

In [ ]:
#Total words in reviews
len(reviews_combined)

In [ ]:
reviews_combined[:200]

In [ ]:
word_cloud = WordCloud().generate_from_text(reviews_combined)

In [ ]:
#?WordCloud

In [ ]:
plt.figure()
plt.imshow(word_cloud)
plt.show()

In [ ]:
word_cloud = WordCloud(width=800,height=800,background_color='white',max_words=150).\
generate_from_text(reviews_combined)

In [ ]:
plt.figure(figsize=[8,8])
plt.imshow(word_cloud)
plt.show()

In [ ]:
all_terms = []
fdist = {}
all_terms = reviews_combined.split(" ")
for word in all_terms:
    fdist[word] = fdist.get(word,0) + 1

In [ ]:
freq = {"words":list(fdist.keys()),"freq":list(fdist.values())}
df_dist = pd.DataFrame(freq)

In [ ]:
%matplotlib inline
df_dist.sort_values(ascending=False, by="freq").head(25).\
plot.bar(x= "words", y= "freq",figsize=(20,5)) 

Problems with the above visuals
    1.case is non uniform
    2. punctuations and stop words
    3. words with different tense forms like charged, charging, charges has its root form in charge

In [ ]:
#1. case normalization and tokenizing

In [ ]:
reviews.review[:10]

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
#All Reviews tokenized and in lower case
all_terms = word_tokenize(reviews_combined.lower())

In [ ]:
print(all_terms[:20])

In [ ]:
len(set(all_terms))

In [ ]:
#visualizing the frequency distrinution

In [ ]:
from nltk.probability import FreqDist

In [ ]:
fdist = FreqDist(all_terms)
fdist

In [ ]:
fdist.plot(30,cumulative=False)
plt.show()

In [ ]:
from string import punctuation
from nltk.corpus import stopwords

In [ ]:
stop_nltk = stopwords.words("english")

In [ ]:
stop_updated = stop_nltk + list(punctuation) + ["..."] + ["phone", "mobile","lenovo","k8","product"]

below code removes:
1. stopwords, 
2. punct's,
3. custom stop words,
4. words with lenght less than 2

In [ ]:
reviews_updated1 = [word for word in all_terms if word not in stop_updated and len(word) > 2]

In [ ]:
print(reviews_updated1[:20])

In [ ]:
len(set(reviews_updated1))

In [ ]:
from nltk.stem import SnowballStemmer
stemmer_s = SnowballStemmer("english")

In [ ]:
reviews_updated_stem = [ stemmer_s.stem(word) for word in reviews_updated1]

In [ ]:
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

In [ ]:
reviews_updated_lemm = [ lemm.lemmatize(word) for word in reviews_updated1]

In [ ]:
print(len(set(reviews_updated_stem)),len(set(reviews_updated_lemm)))

In [ ]:
#Stememr reduces the number of distinct words in the corpus to a greater exten

In [ ]:
# Define function that takes in a raw string, returns a clean string
# - Tokenize, lower case, remove stop words, stem, stitch back to string
def clean_txt(sent):
    tokens = word_tokenize(sent.lower())
    stemmed = [stemmer_s.stem(term) for term in tokens if term not in stop_updated and len(term) > 2] 
    res = " ".join(stemmed)
    return res

In [ ]:
reviews['clean_review'] = reviews.review.apply(clean_txt)

In [ ]:
reviews.head()

# Word cloud on cleaned dataset

In [ ]:
reviews_combined = " ".join(reviews.clean_review.values)

In [ ]:
word_cloud = WordCloud(width=800,height=800,background_color='white',max_words=150).\
generate_from_text(reviews_combined)

In [ ]:
plt.figure(figsize=[8,8])
plt.imshow(word_cloud)
plt.show()

# Bar Graph on top 25 words

In [ ]:
all_terms = []
fdist = {}
all_terms = reviews_combined.split(" ")
for word in all_terms:
    fdist[word] = fdist.get(word,0) + 1

In [ ]:
freq = {"words":list(fdist.keys()),"freq":list(fdist.values())}
df_dist = pd.DataFrame(freq)

In [ ]:
%matplotlib inline
df_dist.sort_values(ascending=False, by="freq").head(25).\
plot.bar(x= "words", y= "freq",figsize=(20,5)) 

# Plot the bar graph for top 25 frequenctly used bigrams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# create a bigram count vectorizer object
bigram_count_vectorizer = CountVectorizer(ngram_range=(2,2),max_features = 1500)

In [ ]:
X_bigram = bigram_count_vectorizer.fit_transform(reviews['clean_review'])

# Creating a DTM
DTM_bigram = pd.DataFrame(X_bigram.toarray(), columns=bigram_count_vectorizer.get_feature_names())

In [ ]:
bigram_count_vectorizer.get_feature_names()[:20]

In [ ]:
DTM_bigram.head()

In [ ]:
DTM_bigram.sum().sort_values(ascending=False).head(25).plot.bar(figsize=(20,5)) 

In the amazon case study dataset
Find the top 5 relavent reviews which has the below words
 - "battery drain"
 -"Good camera"
If there is a clash in the scores, review at index 0 has higher precedence than at index 1 and so on....

# Sentiment Classification

### Sentiment Prediction Strategy 1: Word based scoring 

In [ ]:
afinn = pd.read_csv("AFINN-111.txt", sep="\t", header=None)
afinn.columns = ['Term','Score']
term_scores = dict(afinn.values)

In [ ]:
def get_sentiment(sent):
    tokens = word_tokenize(sent.lower())
    score = 0
    for term in tokens:
        score += term_scores.get(term,0)
    return score

In [ ]:
reviews['sent_score'] = reviews.clean_review.apply(get_sentiment)

In [ ]:
reviews.head()

In [ ]:
reviews.sent_score.hist()

#sentiment Prediction
    if the sent_score > 0 predicted sentiment is 1
    else the predicted sentiment is 0

In [ ]:
reviews['sent_prediction'] = reviews.sent_score.apply(lambda x:1 if x>0 else 0)

In [ ]:
reviews.head()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(reviews.sentiment,reviews.sent_prediction)

In [ ]:
accuracy_score(reviews.sentiment,reviews.sent_prediction)

In [ ]:
#lets build a udf which takes a text as input and does all the above cleaning steps
# one review at a time will be passed from the the reviews column to this function
#based on the outcome of this function, you should be able to add a new column in the data frame
def  clean_txt_new(sent):
    tokens = word_tokenize(sent.lower().replace('[^a-z ]',' '))
    cleaned_token = [(term) for term in tokens 
               if term not in stop_updated and len(term) > 2] 
    res = " ".join(cleaned_token)
    return res

In [ ]:
reviews['clean_review_new'] = reviews.review.apply(clean_txt_new)

In [ ]:
reviews.head()

In [ ]:
reviews['sent_score_new'] = reviews.clean_review_new.apply(get_sentiment)

In [ ]:
reviews['sent_prediction_new'] = reviews.sent_score_new.apply(lambda x:1 if x>0 else 0)

In [ ]:
accuracy_score(reviews.sentiment,reviews.sent_prediction_new)

### Sentiment Prediction Strategy 2: Using popular package - Vader

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
def get_vader_sentiment(sent):
    return analyser.polarity_scores(sent)['compound']

In [ ]:
reviews['score_vader'] = reviews.review.apply(get_vader_sentiment)

In [ ]:
reviews['sent_prediction_vader'] = reviews.score_vader.\
apply(lambda x:1 if x>0 else 0)

In [ ]:
accuracy_score(reviews.sentiment,reviews.sent_prediction_vader)

In [ ]:
confusion_matrix(reviews.sentiment,reviews.sent_prediction_vader)

### Sentiment Prediction Strategy : Making our own model

In [ ]:
reviews.head()

In [ ]:
X = reviews.clean_review.values
y = reviews.sentiment.values

In [ ]:
X[:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,
                                                 random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(max_features=1500)

In [ ]:
# Extarct the features and transform -> get the count of terms in every document
X_train_bow = vectorizer.fit_transform(X_train)

In [ ]:
# Only transform -> get the count of terms in every document, the test data

X_test_bow = vectorizer.transform(X_test)

In [ ]:
X_train_bow.shape, X_test_bow.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train_bow,y_train)

In [ ]:
y_test_pred = logreg.predict(X_test_bow)

In [ ]:
accuracy_score(y_test, y_test_pred)

#### Sentiment classification using tfidfvectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features=2000)

In [ ]:
# Extarct the features and transform -> get the count of terms in every document
X_train_bow = vectorizer.fit_transform(X_train)

In [ ]:
# Only transform -> get the count of terms in every document, the test data
X_test_bow = vectorizer.transform(X_test)

In [ ]:
X_train_bow.shape, X_test_bow.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train_bow,y_train)

In [ ]:
y_test_pred = logreg.predict(X_test_bow)

In [ ]:
accuracy_score(y_test, y_test_pred)

### Document clustering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvectorizer = TfidfVectorizer(max_features=150)

In [ ]:
X = tfidfvectorizer.fit_transform(reviews['clean_review'])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
num_clusters = 6
km = KMeans(n_clusters=num_clusters)
km.fit(X)

In [ ]:
pred_cluster = km.predict(X)
reviews['cluster_predicted'] = pred_cluster
reviews['cluster_predicted'].value_counts()

In [ ]:
pred_cluster.shape

In [ ]:
km.cluster_centers_.shape

in TFIDF or in DTM every word is a feature
when these words are clustered, the cluster center value for every feature tells us the centeroid of the word relavencies/frequencies.

Higher the cluster centeroid for that feature, higher will be value of word relevency/frequency.

In [ ]:
print("Top 5 terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidfvectorizer.get_feature_names()
for i in range(num_clusters):
    print ("Cluster %d:" % i),
    for ind in order_centroids[i, : 5]:
        print (' %s' % terms[ind]),
    print

In [ ]:
group = reviews[reviews['cluster_predicted'] == 1]
text = ' '.join(reviews['clean_review'])
wc = WordCloud(background_color='white').generate(text)
plt.imshow(wc)

### Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfidfvectorizer = TfidfVectorizer(max_features=1500)

In [ ]:
X = tfidfvectorizer.fit_transform(reviews['clean_review'])
df_tfidf = pd.DataFrame(X.toarray(),columns=tfidfvectorizer.get_feature_names())

In [ ]:
#Check the cosine similarity with the second and the third reviews
cosine_similarity(df_tfidf[1:2], df_tfidf[2:3])

In [ ]:
cosine_similarity([df_tfidf['batteri'],df_tfidf['battery']])

In [ ]:
cs = cosine_similarity(df_tfidf.T)

In [ ]:
def get_similar_words(input_word, sim_mat):
    cos_vals = sim_mat[input_word].sort_values(ascending=False)
    similar_words = cos_vals.drop(input_word).head(5)
    return similar_words

In [ ]:
sim_mat = cosine_similarity(df_tfidf.T)

sim_mat = pd.DataFrame(sim_mat, columns= df_tfidf.columns, index= df_tfidf.columns)

get_similar_words('drain', sim_mat)

In [ ]:
#Understanding how the function works
sim_mat['charge'].sort_values(ascending=False).drop('charge').head(5)